# 2.0 CCLE Gene Expression Signature

In [1]:
from clustergrammer2 import net

clustergrammer2 backend version 0.2.9


In [2]:
import clustergrammer_groupby as cby
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
df = pd.read_csv('../data/CCLE/CCLE.txt.gz', compression='gzip', index_col=0)

In [4]:
net.load_df(df)
net.filter_N_top(inst_rc='row', N_top=1000, rank_type='var')
net.normalize(axis='row', norm_type='zscore')
df_var = net.export_df().round(2)
genes_top_var = df_var.index.tolist()

In [5]:
df_var.index.tolist()[0]

'KRT19'

In [6]:
len(genes_top_var)

1000

In [7]:
net.load_df(df)
net.filter_N_top(inst_rc='row', N_top=1000, rank_type='var')
net.normalize(axis='row', norm_type='zscore')
df_tmp = net.export_df().round(2)
net.enrichrgram('GO_Biological_Process_2018')
df_enr = net.export_df()
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "KRT19", "ini": 1000, "clust": 967, "rank": 241, "rankvar": 258…

In [8]:
df_enr.index.tolist()[0]

('KRT19',
 'extracellular matrix organization (GO:0030198): False<p> Pval 9.171026638093131e-29</p>',
 'collagen fibril organization (GO:0030199): False<p> Pval 1.9897240254393085e-12</p>',
 'regulation of cell migration (GO:0030334): False<p> Pval 6.6999547584764606e-15</p>',
 'positive regulation of intracellular signal transduction (GO:1902533): False<p> Pval 5.5852790328680665e-12</p>',
 'negative regulation of cellular process (GO:0048523): False<p> Pval 1.205623445289199e-13</p>',
 'negative regulation of cell proliferation (GO:0008285): False<p> Pval 2.3244136779160615e-10</p>',
 'neutrophil mediated immunity (GO:0002446): False<p> Pval 7.30336780471453e-10</p>',
 'cytokine-mediated signaling pathway (GO:0019221): False<p> Pval 1.2455747717025743e-13</p>',
 'neutrophil degranulation (GO:0043312): False<p> Pval 2.8513592127176324e-09</p>',
 'regulation of angiogenesis (GO:0045765): False<p> Pval 2.3387050175471637e-10</p>')

In [9]:
enrichrgram_row = df_enr.index.tolist()[0][1:]
enrichrgram_row = [x.split(': ')[0] + ' '+  x.split('Pval')[1] for x in enrichrgram_row]
enrichrgram_row

['extracellular matrix organization (GO:0030198)  9.171026638093131e-29</p>',
 'collagen fibril organization (GO:0030199)  1.9897240254393085e-12</p>',
 'regulation of cell migration (GO:0030334)  6.6999547584764606e-15</p>',
 'positive regulation of intracellular signal transduction (GO:1902533)  5.5852790328680665e-12</p>',
 'negative regulation of cellular process (GO:0048523)  1.205623445289199e-13</p>',
 'negative regulation of cell proliferation (GO:0008285)  2.3244136779160615e-10</p>',
 'neutrophil mediated immunity (GO:0002446)  7.30336780471453e-10</p>',
 'cytokine-mediated signaling pathway (GO:0019221)  1.2455747717025743e-13</p>',
 'neutrophil degranulation (GO:0043312)  2.8513592127176324e-09</p>',
 'regulation of angiogenesis (GO:0045765)  2.3387050175471637e-10</p>']

In [10]:
df.shape

(18874, 1037)

In [11]:
from ast import literal_eval as make_tuple
cols = df.columns.tolist()
new_cols = [make_tuple(x) for x in cols]
df.columns = new_cols

In [12]:
all_genes = df.index.tolist()
len(all_genes)

18874

# CCLE Gene Expression Data, Z-score Genes

In [13]:
rows = df.index.tolist()

In [14]:
rows[0]

'LOC100009676'

In [15]:
rows = df.index.tolist()
new_rows = [(x,) for x in rows]
df.index = new_rows
net.load_df(df)
net.filter_N_top(inst_rc='row', N_top=1000, rank_type='var')
net.normalize(axis='row', norm_type='zscore')
df_tmp = net.export_df().round(2)
net.load_df(df_tmp)
net.cluster()
net.dendro_cats(axis='row', dendro_level=5)
df_dendro = net.export_df()
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "KRT19", "ini": 1000, "clust": 967, "rank": 948, "rankvar": 689…

In [16]:
rows = df_dendro.index.tolist()
rows[0]

('KRT19', 'Group 5: cat-13')

In [17]:
gene_list = sorted([x[0] for x in rows if x[1] == 'Group 5: cat-5'])
len(gene_list)

121

In [18]:
gmts = {}
gmts['kea'] = net.load_gmt('../data/Enrichr_Libraries_of_Interest/KEA_2015.txt')
gmts['go-process'] = net.load_gmt('../data/Enrichr_Libraries_of_Interest/GO_Biological_Process_2018.txt')

In [19]:
len(all_genes)

18874

In [20]:
# len(set(all_genes).intersection(gene_list))

In [21]:
from scipy.stats import binom_test
def enrich_gene_list_using_lib(lib_json, gene_list, background_list, pval_cutoff=0.05):
    len_gene_list = len(gene_list)
    
    list_terms = []
    list_pval = []
    # list of series that will be used to make dataframe
    list_term_ser = []
    for inst_term in lib_json:
        
        term_ser = pd.Series(data=np.zeros(len(gene_list)), index=gene_list)
        
        term_genes = lib_json[inst_term]
        
        p_expect = len(set(all_genes).intersection(term_genes))/len(all_genes)
        
#         print(gene_list)

        found_genes = list(set(gene_list).intersection(term_genes))
        
#         print('found_genes', len(found_genes))
        actual_k = len(found_genes)
        
        # set found genes to one
        term_ser[found_genes] =  1
        
        if actual_k/len_gene_list > p_expect:
            inst_pval = binom_test(actual_k, len_gene_list, p_expect)
        else:
            inst_pval = 0.5
        
#         print('HERE', inst_pval, actual_k, len_gene_list, p_expect)
        
        if inst_pval < pval_cutoff:

            term_name = (inst_term, 'Pval: ' + str(inst_pval))
            
            list_terms.append(term_name)
            list_pval.append(inst_pval)

            term_ser.name = term_name

            list_term_ser.append(term_ser)
            
    ser_pval = pd.Series(data=list_pval, index=list_terms).sort_values()
    df_term = pd.concat(list_term_ser, axis=1)
    
    # remove genes (rows) with no matches
    gene_sum = df_term.sum(axis=1)
    keep_genes = gene_sum[gene_sum > 0].index.tolist()
    df_term = df_term.loc[keep_genes]
    
    return ser_pval, df_term

In [22]:
from copy import deepcopy

In [23]:
def enrich_dataframe_using_lib(lib_json, df_ini, background_list, num_top_terms, pval_cutoff=0.05):

    gene_list = df_ini.index.tolist()
    
    ser_pval, df_term = enrich_gene_list_using_lib(lib_json, gene_list, background_list, 
                                                   pval_cutoff=pval_cutoff)
    
    keep_terms = ser_pval.index.tolist()[:num_top_terms]

    df_term = df_term[keep_terms]
    
    # drop pval, going to use the gmt json to add categories to rows
    keep_term_names = [x[0] for x in keep_terms]
    
    rows_ini = df_ini.index.tolist()
    
    new_rows = []
    for inst_gene in rows_ini:
        new_row = (inst_gene,)
    
        for inst_term in keep_terms:
            
            inst_term_name = inst_term[0]
            inst_term_pval = inst_term[1]
            
            term_list = lib_json[inst_term_name]
            
            inst_found = 'False'
            if inst_gene in term_list:
                inst_found = 'True'
                
            inst_cat = inst_term_name + ': ' + inst_found + '<p> ' + inst_term_pval + '</p>'
            new_row = new_row + (inst_cat,)

        new_rows.append(new_row)
        
    df_cat = deepcopy(df_ini)
    df_cat.index = new_rows
    
    return df_cat, ser_pval, df_term

In [24]:
df_tmp.shape

(1000, 1037)

In [25]:
len(all_genes)

18874

In [26]:
df_var.index.tolist()[0]

'KRT19'

In [27]:
df_cat, ser_pval, df_term = enrich_dataframe_using_lib(gmts['go-process'], df_var, all_genes, 10)

In [28]:
df_cat.shape

(1000, 1037)

# Compare Local vs Enrichrgram Enrichment

In [29]:
df_cat.index.tolist()[0]

('KRT19',
 'extracellular matrix organization (GO:0030198): False<p> Pval: 4.88614872976e-26</p>',
 'cytokine-mediated signaling pathway (GO:0019221): False<p> Pval: 3.32810552013e-15</p>',
 'regulation of cell migration (GO:0030334): False<p> Pval: 1.5209499587e-14</p>',
 'regulation of cell proliferation (GO:0042127): False<p> Pval: 7.23143833268e-14</p>',
 'cellular response to cytokine stimulus (GO:0071345): False<p> Pval: 1.19559734474e-13</p>',
 'negative regulation of cellular process (GO:0048523): False<p> Pval: 3.10549303206e-13</p>',
 'positive regulation of cell proliferation (GO:0008284): False<p> Pval: 3.78251598995e-11</p>',
 'collagen fibril organization (GO:0030199): False<p> Pval: 3.95949893332e-11</p>',
 'positive regulation of intracellular signal transduction (GO:1902533): False<p> Pval: 6.85898688745e-11</p>',
 'skin development (GO:0043588): False<p> Pval: 1.24056102141e-10</p>')

In [30]:
df_enr.index.tolist()[0]

('KRT19',
 'extracellular matrix organization (GO:0030198): False<p> Pval 9.171026638093131e-29</p>',
 'collagen fibril organization (GO:0030199): False<p> Pval 1.9897240254393085e-12</p>',
 'regulation of cell migration (GO:0030334): False<p> Pval 6.6999547584764606e-15</p>',
 'positive regulation of intracellular signal transduction (GO:1902533): False<p> Pval 5.5852790328680665e-12</p>',
 'negative regulation of cellular process (GO:0048523): False<p> Pval 1.205623445289199e-13</p>',
 'negative regulation of cell proliferation (GO:0008285): False<p> Pval 2.3244136779160615e-10</p>',
 'neutrophil mediated immunity (GO:0002446): False<p> Pval 7.30336780471453e-10</p>',
 'cytokine-mediated signaling pathway (GO:0019221): False<p> Pval 1.2455747717025743e-13</p>',
 'neutrophil degranulation (GO:0043312): False<p> Pval 2.8513592127176324e-09</p>',
 'regulation of angiogenesis (GO:0045765): False<p> Pval 2.3387050175471637e-10</p>')

# Local Enrichment

In [31]:
net.load_df(df_cat)
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "KRT19", "ini": 1000, "clust": 967, "rank": 948, "rankvar": 689…

In [ ]:
df_term.shape

In [ ]:
genes_1k = [x[0] for x in rows]

# KEA

In [ ]:
ser_pval, df_term = enrich_gene_list_using_lib(gmts['kea'], genes_1k, all_genes)

In [ ]:
keep_top_enr = ser_pval.index.tolist()[:10]

In [ ]:
net.load_df(df_term[keep_top_enr])
net.widget()

# GO Biological Process

In [ ]:
rows[:10]

In [ ]:
len(all_genes)

In [ ]:
len(genes_top_var)

In [ ]:
ser_pval, df_term = enrich_gene_list_using_lib(gmts['go-process'], genes_top_var, all_genes, 0.01)
df_term.shape

#### Terms from Enrichrgram

In [ ]:
enrichrgram_row

#### Locally Enriched Terms

In [ ]:
keep_top_enr = ser_pval.index.tolist()[:10]
keep_top_enr = [x[0] + ' ' + x[1] for x in keep_top_enr]
keep_top_enr

In [ ]:
net.load_df(df_term[keep_top_enr])
net.widget()

In [ ]:
len(gene_list)

In [ ]:
df_sig, keep_genes_dict, df_gene_pval, fold_info = cby.generate_signatures(df,
                                                                     'tissue', num_top_dims=100)

net.load_df(df_sig)
net.widget()

### Add tissue category to genes

In [ ]:
gene_sig = df_sig.idxmax(axis=1)
gs_dict = {}
for inst_gene in gene_sig.index.tolist():
    gs_dict[inst_gene] = gene_sig[inst_gene][0]
    
rows = df.index.tolist()
new_rows = [(x, 'Cell Type: ' + gs_dict[x]) if x in gs_dict else (x, 'N.A.') for x in rows ]
df.index = new_rows
net.load_df(df)

In [ ]:
ct_color = net.viz['cat_colors']['col']['cat-0']

In [ ]:
def set_cat_colors(axis, cat_index, cat_title=False):
    for inst_ct in ct_color:
        if cat_title != False:
            cat_name = cat_title + ': ' + inst_ct
        else:
            cat_name = inst_ct
            
        inst_color = ct_color[inst_ct]
        net.set_cat_color(axis=axis, cat_index=cat_index, cat_name=cat_name, inst_color=inst_color)

In [ ]:
set_cat_colors('row', 1)

### CCLE Data with Gene-Tissue Category

In [ ]:
net.load_df(df)
net.widget()